In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import os
import pandas as pd
from scipy.io import arff

from autogluon.tabular import TabularPredictor, TabularDataset

/home/danish/anaconda3/envs/AutoML/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import os
import pandas as pd

# Path to the directory containing your CSV files

directory_path = '/home/danish/Datasets/EVSE-B'
# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Loop through each CSV file and concatenate its data to the combined DataFrame
for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)
    df = pd.read_csv(file_path).dropna()
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Display the combined DataFrame
data = combined_df # .sample(frac=0.5, random_state=42)
data.shape
# Optionally, you can save the combined DataFrame to a new CSV file
# combined_df.to_csv('/path/to/combined_data.csv', index=False)

(115298, 10)

In [3]:
data.columns.to_list()

['time',
 'shunt_voltage',
 'bus_voltage_V',
 'current_mA',
 'power_mW',
 'State',
 'Attack',
 'Attack-Group',
 'Label',
 'interface']

In [6]:
data = data[['shunt_voltage', 'bus_voltage_V', 'current_mA', 'power_mW', 'Attack-Group']]
data['Attack-Group'].value_counts()


Attack-Group
DoS            34884
recon          33318
host-attack    32733
none           14363
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

data = TabularDataset(data)

# Define the target column name
target_column = 'Attack-Group'

# Split data into training and validation sets (e.g., 80% train, 20% validation)
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [10]:

# Initialize AutoGluon TabularPredictor
predictor = TabularPredictor(label=target_column, path='/home/danish/Codes/AutoML/Gluon_EVSE-B/')  # Specify path to save models


In [11]:
predictor.fit(train_data=train_data, time_limit=3600)  # Set time_limit (in seconds) for training


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "/home/danish/Codes/AutoML/Gluon_EVSE-B/"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #41~22.04.2-Ubuntu SMP PREEMPT_DYNAMIC Mon Jun  3 11:32:55 UTC 2
CPU Count:          128
Memory Avail:       48

[1000]	valid_set's multi_error: 0.3564


	0.6444	 = Validation score   (accuracy)
	5.05s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 3545.42s of the 3545.42s of remaining time.
	0.6552	 = Validation score   (accuracy)
	1.03s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ... Training model for up to 3544.37s of the 3544.37s of remaining time.
	0.6336	 = Validation score   (accuracy)
	1.1s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestEntr ... Training model for up to 3541.75s of the 3541.75s of remaining time.
	0.6356	 = Validation score   (accuracy)
	1.31s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 3539.01s of the 3539.01s of remaining time.
	0.6464	 = Validation score   (accuracy)
	2.13s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ... Training model for up to 3536.86s of the 3536.86s of remaining time.


In [12]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# labels = le.fit_transform(val_data['marker'])
# labels

# # print(y_true)
# # print(y_pred)

# val_data['marker'] = labels

# Evaluate the trained model on the validation set
performance = predictor.evaluate(val_data)

# Display evaluation metrics
print("Validation Performance:")
print(performance)

Validation Performance:
{'accuracy': 0.6506071118820468, 'balanced_accuracy': 0.6708131821906921, 'mcc': 0.5402363424675645}
